In [28]:
!kaggle competitions download -c home-data-for-ml-course -p .


home-data-for-ml-course.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import numpy as np
import pandas as pd
import zipfile


In [30]:
with zipfile.ZipFile("home-data-for-ml-course.zip", "r") as zip_ref:
    zip_ref.extractall(".")  


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(columns=['Id'],inplace=True)
test.drop(columns=['Id'],inplace=True)

#fillna with 0 and None

train[train.select_dtypes(include=['number']).columns] = train.select_dtypes(include=['number']).fillna(0)
test[test.select_dtypes(include=['number']).columns] = test.select_dtypes(include=['number']).fillna(0)

train.select_dtypes(include=['object']).fillna('None', inplace=True)
test.select_dtypes(include=['object']).fillna('None', inplace=True)

numeric_cols = train.select_dtypes(include=['number']).columns
train[numeric_cols] = np.log1p(train[numeric_cols])

numeric_cols = test.select_dtypes(include=['number']).columns
test[numeric_cols] = np.log1p(test[numeric_cols])

print(train.shape) 
print(test.shape)   

(1460, 80)
(1459, 79)


In [33]:
#one hot encoding for categorical columns
train = pd.get_dummies(train).astype(float)
test = pd.get_dummies(test).astype(float)
X_train = train.drop(columns=['SalePrice'])
y_train = train['SalePrice']  

X_train, X_test = X_train.align(test, join='left', axis=1, fill_value=0)

X_train = X_train.values
y_train = y_train.values
X_test = X_test.values


In [ ]:
print(X_train.shape)  
print(X_test.shape)   

(1460, 287)
(1459, 287)


In [35]:
def OLS(X,y, lambda_reg=1.0):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]  # Add bias term
    n_features = X_b.shape[1]
    
    # Identity matrix for regularization (excluding bias term)
    I = np.eye(n_features)
    I[0, 0] = 0  # Do not regularize the bias term

    # Compute theta using Ridge Regression
    theta = np.linalg.inv(X_b.T.dot(X_b) + lambda_reg * I).dot(X_b.T).dot(y)
    return theta


In [36]:
theta=OLS(X_train,y_train)
print(theta)
X_test_b = np.c_[np.ones((X_test.shape[0], 1)), X_test]
y_pred = X_test_b.dot(theta)
y_pred=np.exp(y_pred)

[ 5.52629872e+00 -2.55410344e-02 -2.75464963e-03  7.97157096e-02
  2.45430898e-01  2.30997001e-01  7.24669371e-02  8.35010508e-02
 -1.00313838e-03  1.07394367e-02 -6.23474500e-03 -5.56715733e-03
  5.60266900e-02  3.58561163e-02  1.57574968e-03 -7.55549288e-03
  3.69283931e-01  3.46783434e-02  7.95224273e-03  7.00266546e-02
  4.24680913e-02 -6.29260584e-03 -1.07196052e-01  4.28761268e-02
  2.42142441e-02 -1.44317175e-02  1.04464657e-01  1.78086256e-02
  4.12002480e-03  1.95930455e-03  8.32172332e-04  4.92516249e-03
  8.87905997e-03  1.49154179e-02 -4.98781256e-03  2.81330453e-03
 -2.65686568e-03 -2.94269257e-01  9.49750469e-02  8.14344546e-02
  7.06046842e-02  4.72550716e-02 -4.59303379e-02  4.59303379e-02
 -9.70351428e-04  3.81879675e-02 -2.48914419e-04  1.71413453e-02
 -2.79198014e-02  1.10273706e-02 -3.20886294e-02  3.00942836e-02
 -1.26368445e-02  1.46311902e-02  8.49150844e-02 -8.49150843e-02
  2.41116682e-02  4.63422104e-02 -2.60183460e-02 -5.44847325e-02
  1.00492000e-02  7.66759

In [ ]:
submission = pd.DataFrame({'Id': test.index + 1461, 'SalePrice': y_pred})
submission.to_csv('submission.csv', index=False)

### Screenshot here: https://github.com/phuongwhuynh/NLP_Lab/blob/main/Lab67/hw1/sceenshot_hw1.png